In [1]:
!uv pip install datasets

Using Python 3.10.18 environment at: /mnt/c/Users/LANDSOFT/Documents/dev/law/.venv
Audited 1 package in 239ms


In [ ]:
from pathlib import Path
import zipfile
import json
from datasets import Dataset
from tqdm.auto import tqdm

/mnt/c/Users/LANDSOFT/Documents/dev/law/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BASE_DIR = "/mnt/d/data/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터"

In [4]:
# BASE_DIR 아래 모든 zip file 경로 수집
zip_files = list(Path(BASE_DIR).rglob("*.zip"))
print(f"Found {len(zip_files)} zip files.")
for zip_file in zip_files:
    print(zip_file)

Found 4 zip files.
/mnt/d/data/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/01.원천데이터/TS_02.법률.zip
/mnt/d/data/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.법률.zip
/mnt/d/data/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Validation/01.원천데이터/VS_02.법률.zip
/mnt/d/data/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.법률.zip


In [ ]:
# ZIP 파일 내용 확인
with zipfile.ZipFile(zip_files[0], 'r') as zip_ref:
    file_list = zip_ref.namelist()
    print("ZIP 파일 내부 파일 목록:")
    for file in file_list[:10]:  # 처음 10개만 출력
        print(file)

ZIP 파일 내부 파일 목록:
/LJU000006.txt
/LJU000003.txt
/LJU000020.txt
/LJU000004.txt
/LJU000001.txt
/LJU000008.txt
/LJU000014.txt
/LJU000016.txt
/LJU000007.txt
/LJU000005.txt


In [ ]:
# TL JSON 구조 확인
for zip_file in zip_files:
    if "TL" in zip_file.name or "VL" in zip_file.name:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            json_files = [f for f in zip_ref.namelist() if f.endswith('.json')]
            print(f"\n{zip_file.name}의 JSON 파일: {json_files}")
            
            if json_files:
                with zip_ref.open(json_files[0]) as f:
                    data = json.load(f)
                    print(f"JSON 타입: {type(data)}")
                    if isinstance(data, dict):
                        print(f"Keys: {list(data.keys())[:10]}")
                    elif isinstance(data, list):
                        print(f"리스트 길이: {len(data)}")
                        print(f"첫 번째 항목 샘플: {str(data[0])[:200]}")


TL_02.법률.zip의 JSON 파일: ['/Training_legal.json']
JSON 타입: <class 'dict'>
Keys: ['totalcount', 'data']

VL_02.법률.zip의 JSON 파일: ['/Validation_legal.json']
JSON 타입: <class 'dict'>
Keys: ['totalcount', 'data']


In [7]:
# JSON 데이터 구조 상세 확인
for zip_file in zip_files:
    if "TL" in zip_file.name:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            json_files = [f for f in zip_ref.namelist() if f.endswith('.json')]
            with zip_ref.open(json_files[0]) as f:
                json_data = json.load(f)
                print(f"totalcount: {json_data['totalcount']}")
                print(f"data 길이: {len(json_data['data'])}")
                print(f"첫 번째 항목 타입: {type(json_data['data'][0])}")
                print(f"첫 번째 항목 샘플:")
                print(json.dumps(json_data['data'][0], ensure_ascii=False, indent=2)[:500])
        break

totalcount: 63704
data 길이: 63704
첫 번째 항목 타입: <class 'dict'>
첫 번째 항목 샘플:
{
  "book_id": "LJU000001",
  "category": "환경/교통법",
  "popularity": 3,
  "keyword": [
    "개발제한구역의 지정 및 관리에 관한 특별조치법",
    "시정명령",
    "개발제한구역법",
    "원심판결을 파기",
    "도시계획법"
  ],
  "text": "이행강제금부과처분취소 (대법원 2013. 12. 12. 선고 2012두20397 판결) 【출전】 판례공보 제434호, 2014년 1월 15일 193페이지 【판시사항】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법상 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 하는지 여부(소극) [2] 개발제한구역의 지정 및 관리에 관한 특별조치법에 의한 이행강제금 부과의 근거가 되는 시정명령이 이루어져야 하는 시기(=법률 시행일인 2010. 2. 7. 이후) 【판결요지】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법 제30조


In [ ]:
# 데이터셋 딕셔너리
datasets = {}

for zip_file in tqdm(zip_files, desc="Processing ZIP files"):
    
    # TS, VS: 텍스트 파일들
    if "TS" in zip_file.name or "VS" in zip_file.name:
        texts = []
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            for filename in tqdm(sorted(zip_ref.namelist()), 
                                desc=f"Reading {zip_file.stem}", 
                                leave=False):
                if filename.endswith('.txt'):
                    with zip_ref.open(filename) as f:
                        content = f.read().decode('utf-8')
                        texts.append({"text": content})
        
        dataset = Dataset.from_dict({"text": [item["text"] for item in texts]})
        
        if "TS" in zip_file.name:
            datasets['train_short'] = dataset
            print(f"✓ train_short: {len(dataset):,} 개")
        elif "VS" in zip_file.name:
            datasets['valid_short'] = dataset
            print(f"✓ valid_short: {len(dataset):,} 개")
    
    # TL, VL: JSON 파일 (모든 필드 포함)
    elif "TL" in zip_file.name or "VL" in zip_file.name:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            json_files = [f for f in zip_ref.namelist() if f.endswith('.json')]
            
            if json_files:
                with zip_ref.open(json_files[0]) as f:
                    json_data = json.load(f)
                    
                # 'data' 리스트의 모든 필드 포함
                data_list = json_data['data']
                dataset = Dataset.from_list(data_list)
                
                if "TL" in zip_file.name:
                    datasets['train_long'] = dataset
                    print(f"✓ train_long: {len(dataset):,} 개 (totalcount: {json_data['totalcount']:,})")
                    print(f"  필드: {list(dataset.features.keys())}")
                elif "VL" in zip_file.name:
                    datasets['valid_long'] = dataset
                    print(f"✓ valid_long: {len(dataset):,} 개 (totalcount: {json_data['totalcount']:,})")
                    print(f"  필드: {list(dataset.features.keys())}")

# 최종 결과
print(f"\n=== 최종 결과 ===")
for name, ds in datasets.items():
    print(f"{name}: {len(ds):,} 개, 필드: {list(ds.features.keys())}")

print(f"\n총합: {sum(len(ds) for ds in datasets.values()):,} 개")

# 사용 예시
# datasets['train_short'][0]
# datasets['train_long'][0]
# datasets['valid_short'][0]
# datasets['valid_long'][0]

Processing ZIP files:  25%|██▌       | 1/4 [00:26<01:20, 26.98s/it]

✓ train_short: 63,704 개


Processing ZIP files:  50%|█████     | 2/4 [00:46<00:44, 22.41s/it]

✓ train_long: 63,704 개 (totalcount: 63,704)
  필드: ['book_id', 'category', 'popularity', 'keyword', 'text', 'word_segment', 'publication_ymd', 'NE']


Processing ZIP files:  75%|███████▌  | 3/4 [00:50<00:13, 13.95s/it]

✓ valid_short: 7,963 개


Processing ZIP files: 100%|██████████| 4/4 [00:52<00:00, 13.08s/it]

✓ valid_long: 7,963 개 (totalcount: 7,963)
  필드: ['book_id', 'category', 'popularity', 'keyword', 'text', 'word_segment', 'publication_ymd', 'NE']

=== 최종 결과 ===
train_short: 63,704 개, 필드: ['text']
train_long: 63,704 개, 필드: ['book_id', 'category', 'popularity', 'keyword', 'text', 'word_segment', 'publication_ymd', 'NE']
valid_short: 7,963 개, 필드: ['text']
valid_long: 7,963 개, 필드: ['book_id', 'category', 'popularity', 'keyword', 'text', 'word_segment', 'publication_ymd', 'NE']

총합: 143,334 개


In [16]:
from datasets import DatasetDict

datasets = DatasetDict(datasets)
datasets

DatasetDict({
    train_short: Dataset({
        features: ['text'],
        num_rows: 63704
    })
    train_long: Dataset({
        features: ['book_id', 'category', 'popularity', 'keyword', 'text', 'word_segment', 'publication_ymd', 'NE'],
        num_rows: 63704
    })
    valid_short: Dataset({
        features: ['text'],
        num_rows: 7963
    })
    valid_long: Dataset({
        features: ['book_id', 'category', 'popularity', 'keyword', 'text', 'word_segment', 'publication_ymd', 'NE'],
        num_rows: 7963
    })
})

In [17]:
datasets['train_short'][0]

{'text': '이행강제금부과처분취소 (대법원 2013. 12. 12. 선고 2012두20397 판결) 【출전】 판례공보 제434호, 2014년 1월 15일 193페이지 【판시사항】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법상 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 하는지 여부(소극) [2] 개발제한구역의 지정 및 관리에 관한 특별조치법에 의한 이행강제금 부과의 근거가 되는 시정명령이 이루어져야 하는 시기(=법률 시행일인 2010. 2. 7. 이후) 【판결요지】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법 제30조 제1항, 제30조의2 제1항 및 제2항의 규정에 의하면 시정명령을 받은 후 그 시정명령의 이행을 하지 아니한 자에 대하여 이행강제금을 부과할 수 있고, 이행강제금을 부과하기 전에 상당한 기간을 정하여 그 기한까지 이행되지 아니할 때에 이행강제금을 부과·징수한다는 뜻을 문서로 계고하여야 하므로, 이행강제금의 부과·징수를 위한 계고는 시정명령을 불이행한 경우에 취할 수 있는 절차라 할 것이고, 따라서 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 할 필요는 없다. [2] 구 개발제한구역의 지정 및 관리에 관한 특별조치법(2009. 2. 6. 법률 제9436호로 개정되기 전의 것)이나 개발제한구역의 지정 및 관리에 관한 특별조치법(이하 \'개발제한구역법\'이라 한다)이 제정·시행되기 전의 구 도시계획법(2000. 1. 28. 법률 제6243호로 전부 개정되기 전의 것)은 개발제한구역에서 행위제한을 위반한 자에 대한 시정명령을 정하고 있을 뿐이었으나, 2009. 2. 6. 법률 제9436호로 개발제한구역법을 개정하면서 시정명령을 이행하지 아니한 자에 대한 이행강제금을 정한 개발제한구역법 제30조의2를 신설하는 한편 그 이행강제금 부과의 근거가 되는 시정명령에 관한 제30조를 개정하였는데, 건축물·공작물 등의 철거·폐쇄·개축 또는 이전에 관하여는 시정명령의 요건이나 내용이 변경되었

In [18]:
datasets['train_long'][0]

{'book_id': 'LJU000001',
 'category': '환경/교통법',
 'popularity': 3,
 'keyword': ['개발제한구역의 지정 및 관리에 관한 특별조치법',
  '시정명령',
  '개발제한구역법',
  '원심판결을 파기',
  '도시계획법'],
 'text': '이행강제금부과처분취소 (대법원 2013. 12. 12. 선고 2012두20397 판결) 【출전】 판례공보 제434호, 2014년 1월 15일 193페이지 【판시사항】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법상 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 하는지 여부(소극) [2] 개발제한구역의 지정 및 관리에 관한 특별조치법에 의한 이행강제금 부과의 근거가 되는 시정명령이 이루어져야 하는 시기(=법률 시행일인 2010. 2. 7. 이후) 【판결요지】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법 제30조 제1항, 제30조의2 제1항 및 제2항의 규정에 의하면 시정명령을 받은 후 그 시정명령의 이행을 하지 아니한 자에 대하여 이행강제금을 부과할 수 있고, 이행강제금을 부과하기 전에 상당한 기간을 정하여 그 기한까지 이행되지 아니할 때에 이행강제금을 부과·징수한다는 뜻을 문서로 계고하여야 하므로, 이행강제금의 부과·징수를 위한 계고는 시정명령을 불이행한 경우에 취할 수 있는 절차라 할 것이고, 따라서 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 할 필요는 없다. [2] 구 개발제한구역의 지정 및 관리에 관한 특별조치법(2009. 2. 6. 법률 제9436호로 개정되기 전의 것)이나 개발제한구역의 지정 및 관리에 관한 특별조치법(이하 \'개발제한구역법\'이라 한다)이 제정·시행되기 전의 구 도시계획법(2000. 1. 28. 법률 제6243호로 전부 개정되기 전의 것)은 개발제한구역에서 행위제한을 위반한 자에 대한 시정명령을 정하고 있을 뿐이었으나, 2009. 2. 6. 법률 제9436호로 개발제한구역

In [21]:
datasets.save_to_disk("/mnt/d/data/154.의료, 법률 전문 서적 말뭉치/processed_datasets")

Saving the dataset (1/1 shards): 100%|██████████| 7963/7963 [00:00<00:00, 17474.37 examples/s]


In [27]:
import os

# 각 데이터셋을 별도 config로 업로드
print("train_short 업로드 중...")
datasets['train_short'].push_to_hub(
    "brainer/legal_word_dataset", 
    config_name="short",
    split="train",
    num_proc=os.cpu_count()
)

print("valid_short 업로드 중...")
datasets['valid_short'].push_to_hub(
    "brainer/legal_word_dataset", 
    config_name="short",
    split="validation",
    num_proc=os.cpu_count()
)

print("train_long 업로드 중...")
datasets['train_long'].push_to_hub(
    "brainer/legal_word_dataset", 
    config_name="long",
    split="train",
    num_proc=os.cpu_count()
)

print("valid_long 업로드 중...")
datasets['valid_long'].push_to_hub(
    "brainer/legal_word_dataset", 
    config_name="long",
    split="validation",
    num_proc=os.cpu_count()
)

print("✓ 업로드 완료!")

train_short 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  5.38ba/s]rds/s]
Processing Files (1 / 1): 100%|██████████| 30.8MB / 30.8MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  6.40ba/s]8.22s/ shards]
Processing Files (1 / 1): 100%|██████████| 21.2MB / 21.2MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  9.08ba/s]5.82s/ shards]
Processing Files (1 / 1): 100%|██████████| 11.9MB / 11.9MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  8.41ba/s]4.76s/ shards]
Processing Files (1 / 1): 100%|██████████| 11.8MB / 11.8MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  8.16ba/s]4.42s/ shards]
Processing F

valid_short 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 19.22ba/s]
Processing Files (1 / 1): 100%|██████████| 3.65MB / 3.65MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Processing Files (1 / 1): 100%|██████████| 2.54MB / 2.54MB,  0.00B/s  03<00:51,  3.44s/ shards]
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s    | 2/16 [00:03<00:22,  1.64s/ shards]
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 57.60ba/s]
Processing Files (1 / 1): 100%|██████████| 1.50MB / 1.50MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 52.08ba/s]1.00s/ shards]
Processing Files (1 / 1): 100%|██████████| 1.47MB / 1.47MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 46.26ba/s]1.34 shards/s]
Processing Files (1 / 1): 100%|██████████| 1.48MB / 1.48MB,  0.00

train_long 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.37ba/s]rds/s]
Processing Files (1 / 1): 100%|██████████| 35.2MB / 35.2MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  4.54ba/s]0.04s/ shards]
Processing Files (1 / 1): 100%|██████████| 24.2MB / 24.2MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  6.51ba/s]7.01s/ shards]
Processing Files (1 / 1): 100%|██████████| 13.3MB / 13.3MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  7.33ba/s]6.02s/ shards]
Processing Files (1 / 1): 100%|██████████| 13.2MB / 13.2MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  8.26ba/s]6.04s/ shards]
Processing F

valid_long 업로드 중...


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 31.00ba/s]
Processing Files (1 / 1): 100%|██████████| 4.35MB / 4.35MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 38.09ba/s]3.72s/ shards]
Processing Files (1 / 1): 100%|██████████| 3.00MB / 3.00MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 35.50ba/s]
Processing Files (1 / 1): 100%|██████████| 1.71MB / 1.71MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 33.29ba/s]1.55s/ shards]
Processing Files (1 / 1): 100%|██████████| 1.68MB / 1.68MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 37.24ba/s]1.14s/ shards]
Processing Files (1 / 1): 100%|█

✓ 업로드 완료!


In [ ]:
from datasets import load_dataset

# Short 버전
ds_short = load_dataset("brainer/legal_word_dataset", "short")
# {'train': ..., 'validation': ...}

# Long 버전
ds_long = load_dataset("brainer/legal_word_dataset", "long")
# {'train': ..., 'validation': ...}

# 특정 split만
train_short = load_dataset("brainer/legal_word_dataset", "short", split="train")